# Documentos Brasil 

https://link.lens.org/Kr7ZrPzLlJf: Downloaded JSON files [here](https://drive.google.com/drive/folders/1tnLxHd4Jz2s35OdFlnqXRFv4iUEqZ76h?usp=sharing)

[Google Scholar](http://scholar.google.com/scholar_lookup?&title=WIMP+dark+matter+as+radiative+neutrino+mass+messenger&journal=JHEP&volume=10&publication_year=2013&author=Hirsch,M&author=Lineros,RA&author=Morisi,S&author=Palacio,J&author=Rojas,N&author=Valle,JWF') example 

## Excel original con columnas de Google Scholar adicionadas

### Con  Pandas
Requieres enough RAM

Get the lens.org JSON files

In [2]:
import pandas as pd

In [3]:
br=pd.read_excel('/scratch/colav/Publicacoes-periodicos_2013-2020-Scopus_updated.xlsx')

In [4]:
br.shape

(464361, 58)

In [5]:
ls -lh /scratch/colav/Publicacoes-periodicos_2013-2020-Scopus_updated.xlsx

-rw-r--r-- 1 colav colav 193M Nov  6  2023 /scratch/colav/Publicacoes-periodicos_2013-2020-Scopus_updated.xlsx


In [6]:
br.columns

Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Molecular Sequence Numbers',
       'Chemicals/CAS', 'Tradenames', 'Manufacturers', 'Funding Details',
       'Funding Text 1', 'Funding Text 2', 'Funding Text 3', 'Funding Text 4',
       'Funding Text 5', 'References', 'Correspondence Address', 'Editors',
       'Sponsors', 'Publisher', 'Conference name', 'Conference date',
       'Conference location', 'Conference code', 'ISSN', 'ISBN', 'CODEN',
       'PubMed ID', 'Language of Original Document',
       'Abbreviated Source Title', 'Document Type', 'Publication Stage',
       'Access Type', 'Source', 'EID', 'ID bolsistas', '_id', 'title', 'cites',
       'cites_link', 'profiles', 'bibtex', 'old_title', 'cid'],
      dtype='object')

### Datos crudos de Google Scholar

Se puede uar el identificador `'cid'` anterior para conectar con estás bases de datos

### Con Mongodb
Sólo disco

In [13]:
from pymongo import MongoClient
from bson.objectid import ObjectId
import sys

In [14]:
client = MongoClient()

In [15]:
client = MongoClient('172.19.31.8', 27017)

In [16]:
#client.drop_database('la')

In [17]:
client.list_database_names()

['MA',
 'argentina',
 'argentina_new_scholar',
 'argentina_tmp',
 'books',
 'br',
 'colciencias-mabooks',
 'colciencias-names',
 'colciencias-test',
 'colciencias-test-bk',
 'colciencias-test_books',
 'colombia',
 'colombia_impactu',
 'colombia_udea',
 'covidbsm',
 'inspirehep',
 'la',
 'local',
 'mm_books',
 'scholar_bayron',
 'scholar_colombia_update',
 'scholar_colombia_update_2022',
 'scienti_2021',
 'sds',
 'sds_bk_07112022',
 'sds_old',
 'test-book-single',
 'test-book-single-parse']

In [18]:
db = client['br']

In [19]:
db.list_collection_names()

['stage_cites',
 'stage_cites_invalid',
 'quarantine',
 'stage_invalid',
 'cache_cites_count',
 'error_cites_count',
 'gslookup_not_found',
 'data',
 'cache_cites',
 'stage']

Colección de datos de entrada obtenidos a partir del Excel original para buscar la información de Google Scholar a través de Moai

In [20]:
dt=db.get_collection('data')

In [22]:
dt.count_documents({})

437952

Colección con los metadatos de Google Scholar incluyendo el conteo de citas, el bibtex y los perfiles en Google Scholar de los autores

In [21]:
st=db.get_collection('stage')

In [13]:
st.count_documents({})

418177

Colección con los metadatos de las citas

In [23]:
ct=db.get_collection('stage_cites')

In [24]:
ct.count_documents({})

6139404

### Algunos ejemplos de uso de mongo

In [25]:
i=20201
l=[d for d in st.find({'article_id':str(i)})]

In [26]:
from IPython.display import JSON

In [27]:
d=l[0]

In [30]:
import bson

In [31]:
bson.ObjectId

bson.objectid.ObjectId

In [32]:
type(d['_id'])

bson.objectid.ObjectId

In [33]:
type(d['_id'])==bson.objectid.ObjectId

True

In [34]:
JSON( [ dict(zip(d.keys(),[str(x) if isinstance(d['_id'],bson.objectid.ObjectId) else x for x in d.values()]))] )

<IPython.core.display.JSON object>

Count citations

In [36]:
import time

In [37]:
s=time.time()
n=0
i=0
for d in st.find():
    nc=d.get('cites')
    if nc:
        n=n+nc
    
    if i%10000==0:
        print(str(n).zfill(7),end='\r')
    i=i+1
print('')
print(time.time()-s)

10400657
8.123533487319946
